In [ ]:
!pip install -U torch accelerate transformers bitsandbytes \
             langchain langchain-core langchain-community langchain-classic \
             pypdf faiss-cpu sentence-transformers

In [ ]:
from google.colab import userdata
userdata.get('colabllm')

In [ ]:
!pip install langchain-text-splitters

In [ ]:
import torch
import os
import sys
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
# Corrected import for legacy chain:
from langchain_classic.chains.retrieval_qa.base import RetrievalQA

In [ ]:
!pip install -q bitsandbytes

In [ ]:
pip install torch

In [ ]:
import os
from google.colab import userdata
import torch
from transformers import BitsAndBytesConfig

# --- 1. Hugging Face Authentication (CRITICAL) ---
HF_TOKEN = userdata.get('colabllm')

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HF_TOKEN
os.environ["HUGGING_FACE_HUB_TOKEN"] = HF_TOKEN # <-- This line we just fixed

# --- 2. Configuration Variables ---
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

# 3. 4-bit Quantization Configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

print("✅ Authentication, model_id, and bnb_config variables are defined.")

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import files
import os

print("Please select your PDF file using the dialog box that appears below:")

# This opens the upload widget. 'uploaded' is a dictionary
# where the keys are the filenames.
uploaded = files.upload()

# We assume you only uploaded one file (the PDF).
# We take the first (and only) key in the dictionary to get the filename.
try:
    # Get the name of the file that was uploaded
    uploaded_filename = list(uploaded.keys())[0]

    # Define the official PDF_PATH variable
    PDF_PATH = uploaded_filename

    print(f"\n✅ File successfully uploaded and path defined as: {PDF_PATH}")

except IndexError:
    # Handles the case where the user closes the dialog without selecting a file
    print("\n❌ No file was selected. Please run the cell again and choose a PDF.")

except Exception as e:
    print(f"\n❌ An error occurred during upload: {e}")

In [ ]:
# --- Rerun the FAISS Creation using the new PDF_PATH variable ---
# (You will need to ensure the necessary imports for these components are in the previous cells)

loader = PyPDFLoader(PDF_PATH)
data = loader.load()

# Split the document into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(data)

# Define the local HuggingFace Embeddings model
embedding_model_id = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_id)

# Defines the required 'db' variable (FAISS Vector Store)
db = FAISS.from_documents(docs, embeddings)

print("✅ FAISS ('db') created successfully.")

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# 1. Define Quantization Config (bnb_config)
# This step is crucial for 4-bit inference on your GPU.
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

# 2. Load the Model and Tokenizer
# This is where the 'model' variable is defined!
print(f"Loading Model: {model_id}...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
print("✅ LLM Model and Tokenizer loaded successfully.")

In [ ]:
# --- 1. Define the Retriever ---
# Converts the FAISS Vector Store ('db') into a searchable retriever object.
retriever = db.as_retriever()

# --- 2. Create the QA Chain ---
# Create a Hugging Face pipeline for text generation
pipeline_instance = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    max_new_tokens=512 # Adjust as needed
)

# Wrap the Hugging Face pipeline with LangChain's HuggingFacePipeline
llm_pipeline = HuggingFacePipeline(pipeline=pipeline_instance)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm_pipeline,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
)

# --- 3. Define Query and Run Chain (This line defines 'result') ---
query = "What factors are explicitly excluded from consideration when employing a person, according to the Nestlé principles?"
print(f"\n❓ Querying LLM with the context-relevant question: {query}")
result = qa_chain.invoke({"query": query})

# --- 4. Print Result & Sources ---
print("\n--- RAG Chain Result ---")
print(result['result'])

# Verify and display the retrieved sources
print("\n--- Source Documents ---")
for doc in result['source_documents']:
    # Use .get() in case the metadata key is missing
    print(f"Source: {doc.metadata.get('source', 'N/A')} (Page: {doc.metadata.get('page', 'N/A')})")
    print(f"Page Content Snippet: {doc.page_content[:200]}...\n")

print("✅ Block 3: RAG Chain executed successfully!")

In [ ]:
# Create a local directory to hold the index files
FAISS_PATH = "faiss_index_nestle"
os.makedirs(FAISS_PATH, exist_ok=True)

# Save the index and the mapping from index to document content
db.save_local(FAISS_PATH)

print(f"✅ FAISS index saved successfully to: /{FAISS_PATH}")

In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install torch accelerate transformers bitsandbytes --upgrade

In [ ]:
!pip install langchain-community

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# multilingual model
# BGE-M3 supports over 100 languages, including English and German
# Good and solid open source choice for rag

MULTILINGUAL_MODEL_NAME = "BAAI/bge-m3"

# model setup arguments
model_kwargs = {'device': 'cuda'} # for my T4 GPU setup
encode_kwargs = {'normalize_embeddings': True}

# Initialize the new multilingual embedding model
print(f"Loading new multilingual embedding model: {MULTILINGUAL_MODEL_NAME}...")
embeddings = HuggingFaceEmbeddings(
    model_name=MULTILINGUAL_MODEL_NAME,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)
print("Mulitlingual model loaded The 'embeddings' variable is now defined.")

In [ ]:
from langchain_community.vectorstores import FAISS
print("Rebuilding FAISS Index with BAAI/bge-m3 embeddings...")
# this will be able to use new 'embeddings' model to create index

db = FAISS.from_documents(docs, embeddings)

print("FAISS Index rebuilt successfully.")

# now im saving new index to disk
FAISS_PATH = "faiss_index_nestle"
db.save_local(FAISS_PATH)
print(f"New multilingual FAISS index saved to: {FAISS_PATH}")

In [ ]:
import torch
import gc

# 1. Delete the partially loaded model object (if it exists)
try:
    if 'model' in locals() or 'model' in globals():
        del model
    if 'tokenizer' in locals() or 'tokenizer' in globals(): # Added tokenizer
        del tokenizer
    if 'pipeline_instance' in locals() or 'pipeline_instance' in globals(): # Added pipeline_instance
        del pipeline_instance
    if 'llm_pipeline' in locals() or 'llm_pipeline' in globals(): # Added llm_pipeline
        del llm_pipeline
    if 'pipe' in locals() or 'pipe' in globals():
        del pipe
except:
    pass

# 2. Force PyTorch to release its cached memory
torch.cuda.empty_cache()

# 3. Trigger Python's garbage collector
gc.collect()

print("✅ GPU Memory cleared. Proceed to the next step.")

In [ ]:
import torch
import os
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Configuration for 4-bit quantization (bnb_config should already be defined)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

# 🛠️ FIX: Define a memory map to explicitly allow offloading to the CPU/Disk
max_memory_mapping = {
    0: "15GiB", # Specify a concrete memory size for the GPU
    "cpu": "15GiB", # Ensure you have enough CPU RAM for this
}

# Ensure the disk offload directory exists
if not os.path.exists("/tmp/offload"):
    os.makedirs("/tmp/offload")


# Load the Model and Tokenizer
print(f"Reloading Mistral Model with Offloading: {model_id}...")
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    max_memory=max_memory_mapping, # Use the corrected max_memory_mapping
    offload_folder="/tmp/offload" # Specify offload folder separately for disk
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

print("✅ LLM Model and Tokenizer loaded successfully. The RAG chain is ready to run.")

In [ ]:
pip install langchain-core

In [ ]:
# Run this cell first and wait for it to finish!
!pip install \
    langchain \
    langchain-core \
    langchain-community \
    pypdf \
    faiss-cpu \
    sentence-transformers \
    torch \
    accelerate \
    transformers \
    bitsandbytes

In [ ]:
# forced german translation
query = "Was sind die Kernprinzipien der Nestlé HR-Richtlinie in Bezug auf die Leistung? Antworte auf Deutsch:"
print(f"\nRunning Query (Forcing German): {query}")

result = qa_chain.invoke({"query": query})

print("\n--- LLM ANSWER (German Output) ---")
print(result['result'])
print("----------------------------------")
print("✅ RAG Chain executed successfully!")